In [16]:
import math

# Advanced Sensor Board

## References and Datasheets

### UKMARSBot

* [Github Repository](https://github.com/ukmars/ukmarsbot/)
* [Mainboard Schematic](datasheets/ukmarsbot-mainboard-V1.3-alpha-schematic.pdf)
* [Basic Sensor Schematic](datasheets/ukmarsbot-wall-sensor-basic-20190815C-schematic.pdf)
* [Advanced Sensor_schematic](datasheets/ukmarsbot-wall-sensor-advanced-V1.2-schematic.pdf)

### Datasheets

* [BPW96B IR Transistor](datasheets/BPW96B_IR_Transistor.pdf)
* [TEFT4300_IR_Transistor](datasheets/TEFT4300_IR_Transistor.pdf)
* [L-7113SF7C IR Illuminator](datasheets/L-7113SF7C_IR_Emitter.pdf)
* [SHF4550 IR Illuminator](datasheets/SHF_4550_IR_Illuminator.pdf)
* [BC337 Switching Transistor](datasheets/BC337-40TA.pdf)

## Introduction

The Advanced Sensor Board has options for IR emmitters / detectors.  Depending on what is chosen different load and current limiting resistors are needed. 

### Symbols

* $V_s$ - Supply voltage
* $V_f$ - Forward voltage drop across the emitter when turned on
* $V_{ce}$ - Forward voltage drop across the active device when turned on
* $I_f$ - Forward current desired, usually max value from the datasheet
* $r$ - Resistor value

### Curent limiting resistors

Current limiting resistors are used to control the maximum current through the emmitters.  The equation to calculate the resistor value is:

$$ 
    r = {(V_s - V_f) \over I_f}
$$

If we are driving the emmitter using an active device (transistor or FET) then we will also need to take into account any voltage drop across the active device ($V_{ce}$) , making the resistor calculation:

$$
    r = {(V_s - (V_f + V_{ce})) \over I_f}
$$   

Initially when breadboarding I will deal with the steady state (continuously on) illumination.  This will give a safe upper limit to the resistor value.  We will then move on to pulsed illumination.  Pulsed illumination is able to handle larger instantaneous currents for a shorter period with non-illumination periods to allow the emitters to cool down between pulses.

Datasheets will give values of $I_f$ for both continuous and pulsed operation with typical parameters, where $t_p$ is the pulse width.

In [2]:
## Emitters

Vs          = 5         # Volts

### L-7113SF7C

l7113_Vf    = 1.4       # Volts
l7113_If    = 100e-3     # Amps  (Modified to higher than steady state current as we are pulsing)

### SHF4550

shf4550_Vf  = 1.5       # Volts
shf4550_If  = 100e-3    # Amps 

## Detectors

### BWP96B

bwp96b_Ica  = 4.5e-3    # Amps nominal (Range 2.5mA - 7.5mA)
bwp96b_Vf   = 0.3       # Volts (Max)

## Calculate resistor values

Peter Harrison's advice for deciding on the emitter current limit and opto-transistor load resistors is as follows:

> In terms of results, you should probably stick with somewhere between 22 and 33 Ohms for the emitter load and start with 1k0 for the detectors. what you are looking for is to have the maximum raw reading from the sensor at about 700-800 when very close to the wall. That allows for some headroom in a brightly lit environment.



### L-7113 - Current limiting resistors

#### No transistor switching

In [3]:
l7113_R = (Vs - l7113_Vf)  / l7113_If
print(f'{l7113_R} Ω')

36.0 Ω


#### With transistor switching

In [4]:
l7113_r = (Vs - (l7113_Vf + bwp96b_Vf)) / l7113_If
print(f'{l7113_r} Ω')

32.99999999999999 Ω


### SHF4550

In [5]:
shf4550_R = (Vs - shf4550_Vf) / shf4550_If
print(f'{shf4550_R} Ω')

35.0 Ω


## Choose limiting resistor values

Resistors from the  Avanced Sensor Board Schematic:

* R11   = 33 Ω
* R1-4  = 120 Ω


In [6]:
l1173_r = 150


## Calculate load resistor values

### BPW96B

In [7]:
bwp96b_R = (Vs - bwp96b_Vf) / bwp96b_Ica
print(f'{round(bwp96b_R, 2)} Ω')

1044.44 Ω


## Choose load resistor values

Choosing load resistor values.  The circa 1 kΩ value assumes that the receiver is saturated with IR, but the L-1173 does not really produce sufficient IR illumination to turn the transistor on fully.  To get increased sensitivity larger load resistors are needed.  From experimentation, values of 6.2 kΩ has been chosen for R5-8.

If there is a lot of ambient light, lower resistor values may be needed, somewhere in the range 1kΩ to 6.2kΩ

In [8]:
bwp96b_R = 6200

## Beamwidths and depression angles

### Beamwidths

In [3]:
bp96b_beamwidth         = 40    # Degrees
l1173_beamwidth         = 20    # Degrees

max_detection_distance  = 20    # cm
sensing_distance        = 14.5  # cm
min_detection_distance  = 3.5   # cm


In [10]:
## beam radius at max detection distance
bwp96b_beam_radius = max_detection_distance * (bp96b_beamwidth / 2) / 360 * 2 * 3.14159
print(f'bwp96b beam radius at {max_detection_distance} cm = {round(bwp96b_beam_radius, 1)} cm')

## beam radius at min detection distance
bwp96b_beam_radius = min_detection_distance * (bp96b_beamwidth / 2) / 360 * 2 * 3.14159
print(f'bwp96b beam radius at {min_detection_distance} cm = {round(bwp96b_beam_radius, 1)} cm')

## beam radius at sensing distance
bwp96b_beam_radius = sensing_distance * (bp96b_beamwidth / 2) / 360 * 2 * 3.14159
print(f'bwp96b beam radius at {sensing_distance} cm = {round(bwp96b_beam_radius, 1)} cm')


bwp96b beam radius at 20 cm = 7.0 cm
bwp96b beam radius at 3.5 cm = 1.2 cm
bwp96b beam radius at 14.5 cm = 5.1 cm


In [11]:
## beam radius at max detection distance
l1173_beam_radius = max_detection_distance * (l1173_beamwidth / 2) / 360 * 2 * 3.14159
print(f'l1173 beam radius at {max_detection_distance} cm = {round(l1173_beam_radius, 1)} cm')

## beam radius at min detection distance
l1173_beam_radius = min_detection_distance * (l1173_beamwidth / 2) / 360 * 2 * 3.14159
print(f'l1173 beam radius at {min_detection_distance} cm = {round(l1173_beam_radius, 1)} cm')

## beam radius at sensing distance
l1173_beam_radius = sensing_distance * (l1173_beamwidth / 2) / 360 * 2 * 3.14159
print(f'l1173 beam radius at {sensing_distance} cm = {round(l1173_beam_radius, 1)} cm')

l1173 beam radius at 20 cm = 3.5 cm
l1173 beam radius at 3.5 cm = 0.6 cm
l1173 beam radius at 14.5 cm = 2.5 cm


### Depression angles

The walls of the maze are 5cm tall.  At distance we want to be hitting the centre of the wall to give the maximum reflected power.

In [12]:
maze_wall_height        = 5        # cm
sensor_board_height     = 2.15     # cm
detector_mount_height   = 0.45     # cm above sensor board
emitter_mount_height    = 1.25     # cm above sensor board

emitter_height = sensor_board_height + emitter_mount_height     # cm
detector_height = sensor_board_height + detector_mount_height   # cm   

print(f'Emitter height = {emitter_height} cm')
print(f'Detector height = {detector_height} cm')

Emitter height = 3.4 cm
Detector height = 2.6 cm


Detector height is very close to the middle of the walls so a level mounting position should be good.

Emitter could do with a depression angle to give best illumination of the walls.  The longer distances need the most power, so aim for wall centre at the 20cm distance.

In [17]:
emitter_depression_height = emitter_height - maze_wall_height / 2
print(f'Emitter depression height = {round(emitter_depression_height, 1)} cm')

emitter_depression_angle = math.atan(emitter_depression_height / max_detection_distance) * 180 / math.pi
print(f'Emitter depression angle = {round(emitter_depression_angle, 1)} degrees')


Emitter depression height = 0.9 cm
Emitter depression angle = 2.6 degrees
